<a href="https://colab.research.google.com/github/Malerion/Archives-Python/blob/main/GeoCoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
L'API Adresse est une API REST qui renvoie les coordonnées géographiques (latitude et longitude) à partir d'une adresse postale française.
Si plusieurs adresses postales (pour cause d'imprécision ou de numéro de rue inexistant) correspondant à la requête sont trouvées, l'API retourne plusieurs coordonnées, avec à chaque fois un "score". Les coordonnées sont données dans l'ordre de score décroissant, vous pouvez donc retenir la première adresse.

In [1]:
# Exécute le code suivant :
import requests
link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155'
r = requests.get(link).json()
r

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [3.993163, 46.00996]},
   'properties': {'label': '728 Route de Villerest 42155 Ouches',
    'score': 0.951900909090909,
    'housenumber': '728',
    'id': '42162_0008_00728',
    'name': '728 Route de Villerest',
    'postcode': '42155',
    'citycode': '42162',
    'x': 776847.88,
    'y': 6546063.21,
    'city': 'Ouches',
    'context': '42, Loire, Auvergne-Rhône-Alpes',
    'type': 'housenumber',
    'importance': 0.47091,
    'street': 'Route de Villerest'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [3.992892, 45.993344]},
   'properties': {'label': 'Route de Villerest 42155 Lentigny',
    'score': 0.7023167532467532,
    'id': '42120_0066',
    'name': 'Route de Villerest',
    'postcode': '42155',
    'citycode': '42120',
    'x': 776850.11,
    'y': 6544217.88,
    'city': 'Lentigny',
    'context': '42, Loire, Auverg

In [2]:
# Les coordonnées sont données dans l'ordre longitude puis latitude
# Ici nous ne sélectionnons que la première adresse (indice 0)
print("première adresse :", r['features'][0])
print("coordonnées de la première adresse :",r['features'][0]['geometry']['coordinates'])
print("score de la première adresse :",r['features'][0]['properties']['score'])

première adresse : {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [3.993163, 46.00996]}, 'properties': {'label': '728 Route de Villerest 42155 Ouches', 'score': 0.951900909090909, 'housenumber': '728', 'id': '42162_0008_00728', 'name': '728 Route de Villerest', 'postcode': '42155', 'citycode': '42162', 'x': 776847.88, 'y': 6546063.21, 'city': 'Ouches', 'context': '42, Loire, Auvergne-Rhône-Alpes', 'type': 'housenumber', 'importance': 0.47091, 'street': 'Route de Villerest'}}
coordonnées de la première adresse : [3.993163, 46.00996]
score de la première adresse : 0.951900909090909


In [ ]:
# Pour plus de facilité, tu peux ajouter une limite sur le nombre d'éléments retournés

link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155'
r = requests.get(link).json()
print("SANS limite, cette adresse renvoie combien de coordonnées ?",len(r['features']))

link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155&limit=1'
r = requests.get(link).json()
print("AVEC limite, cette adresse renvoie combien de coordonnées ?",len(r['features']))

SANS limite, cette adresse renvoie combien de coordonnées ? 2
AVEC limite, cette adresse renvoie combien de coordonnées ? 1


## Comment créer sa requête à l'API
A toi de modifier la chaine de caractère pour créer la bonne URL de requête

In [10]:
# On observe que la requête se compose d'une partie fixe, suivi de l'adresse à chercher
# Une URL ne peut pas comporter de caractère espace " ",
# et il faut éviter si possible d'avoir des caractères spéciaux ou des accents

link_main = 'https://api-adresse.data.gouv.fr/search/?q='
adresse = '728 Route de Villerest, 42155 Ouches'


link = link_main + adresse.replace(' ', '/')# A toi de jouer ici

print(link)

https://api-adresse.data.gouv.fr/search/?q=728/Route/de/Villerest,/42155/Ouches


In [16]:
# Crée ici une fonction qui transforme une adresse postale en URL de requête pour l'API Adresse,
# puis effectue la requête et retourne les coordonnées :

def API_adresse(adresse_postale):
    adresse_modifiee = adresse_postale.replace(' ', '/')

    link_main = 'https://api-adresse.data.gouv.fr/search/?q='
    url_requete = link_main + adresse_modifiee

    return url_requete


In [18]:
# Teste-là ici :
adresse_postale_test = '728 Route de Villerest, 42155 Ouches'
url_api_adresse_test = API_adresse(adresse_postale_test)
print(url_api_adresse_test)

https://api-adresse.data.gouv.fr/search/?q=728/Route/de/Villerest,/42155/Ouches


# DataViz
Les coordonnées peuvent être utilisées sur des outils de visualisation, qu'il s'agisse d'outils de BI (PowerBI, Tableau), ou des bibliothèques de DataViz en Python comme Plotly ou Folium.

Ici nous allons afficher une carte avec Folium.


In [19]:
# Attention, le système latin retourne les coordonnées en (longitude, latitude),
# Alors que le système anglo-saxon est en (latitude, longitude), il faut donc inverser les coordonnées
# Ca tombe bien, vous savez le faire sur une liste avec un "pas" de moins un.
point = r['features'][0]['geometry']['coordinates'][::-1]
point

[46.00996, 3.993163]

In [21]:
# La syntaxe de Folium est très simple, vous commencez par créer une carte centrée sur un point
# Vous pouvez modifier le niveau de zoom par défaut avec l'argument "zoom_start"
import folium
m = folium.Map(location=point,zoom_start=7)
m

In [22]:
# Puis vous pouvez ajouter des points de repère et mettre un commentaire cliquable
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='Un bon restau sur le pouce'
    ).add_to(m)
m

# Challenge
Voici un DataFrame avec des restaurants de Clermont-Ferrand, et leurs adresses respectives. Voici ta mission :
- Créer une nouvelle colonne "coordonnées", qui stockera les coordonnées correspondant à chaque adresse (tu peux t'aider de la fonction que tu as créée précédemment)
- Afficher une carte avec les 4 marqueurs des restaurants. Attention, les restaurants sont très proches, pense à bien régler le niveau de zoom par défaut pour que ce soit bien lisible. Tu peux centrer la carte sur le premier restaurant.

In [25]:
import pandas as pd
restaurants = pd.DataFrame([["Polypode","6 place du Champgil, Clermont-Ferrand, 63000"],
                            ["Jean-Claude Leclerc", "12 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["L'Écureuil", "18 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["Le Saint-Eutrope", "4 rue St-Eutrope, Clermont-Ferrand, 63000"]],
                           columns = ["nom", "adresse"])

restaurants

,nom,adresse
0,Polypode,"6 place du Champgil, Clermont-Ferrand, 63000"
1,Jean-Claude Leclerc,"12 rue St-Adjutor, Clermont-Ferrand, 63000"
2,L'Écureuil,"18 rue St-Adjutor, Clermont-Ferrand, 63000"
3,Le Saint-Eutrope,"4 rue St-Eutrope, Clermont-Ferrand, 63000"


In [34]:
# Ton code ici :
import pandas as pd
import folium
import requests

def API_adresse(adresse_postale):
    adresse_modifiee = adresse_postale.replace(' ', '+')
    link_main = 'https://api-adresse.data.gouv.fr/search/?q='
    url_requete = link_main + adresse_modifiee
    return url_requete

restaurants["coordonnées"] = restaurants["adresse"].apply(API_adresse)
#on récupère les coord du premier restau
premier_restaurant_coords = requests.get(restaurants["coordonnées"].iloc[0]).json()['features'][0]['geometry']['coordinates'][::-1]

#carte centré sur le premier restau
m = folium.Map(location= premier_restaurant_coords, zoom_start=17)
#on pose les marqueurs
for index, row in restaurants.iterrows():
  coords = requests.get(row["coordonnées"]).json()['features'][0]['geometry']['coordinates'][::-1]
  folium.Marker(location=coords, popup=row["nom"]).add_to(m)

m

# Remarques sur l'API Adresse
Comme indiqué dans la quête, il existe un grand nombre de ressources permettant de faire du géocodage. La plupart sont sur inscription, et certaines sont payantes.

L'API Adresse est gratuite et sans inscription. L'inconvénient est qu'elle est relativement lente. Si tu dois l'utiliser à l'avenir, pense donc à stocker les résultats pour éviter de devoir relancer le traitement plusieurs fois.

Pour ta connaissance, il existe également :
- l'API Adresse **reverse** qui permet de trouver l'adresse postale la plus proche à partir de coordonnées géographiques
- l'API à partir d'un **fichier CSV** si vous avez beaucoup de lignes à géocoder
- l'API **découpage administratif** qui permet d'obtenir un format geoJSON des communes et départements pour effectuer des cartes choroplèthes

Toutes la [documentation est disponible ici](https://geo.api.gouv.fr/adresse).